In [1]:
import cantera as ct
import numpy as np
import os
import sys

print('Using the following versions of: ')
print("python",sys.version)
print('numpy',np.__version__)
print('cantera',ct.__version__)
os.getcwd() 

Using the following versions of: 
python 3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
numpy 1.15.4
cantera 2.4.0


'/Users/odiazib/data/GasVerification'

##### This code compares net reaction rate computation of TChem++ with cantera

In [2]:
def runTchem():
    os.chdir(working_dirr) 
    os.system('./delete') # delete old files
    # chem files
    chemfile      = dirInputs + 'ch4NOxRanzi.inp'
    thermfile     = dirInputs + 'therm_ch4NoxRanzi.dat'
    #operating condition files
    inputfile     = dirInputs + 'inputGas.dat'
    #output files
    outputfile    = 'omega.dat'
    
    comand = direx+"TChem_NetProductionRatePerMass.x"+\
              " --chemfile="+ chemfile + \
              " --thermfile="+ thermfile +\
              " --inputfile="+inputfile +\
              " --outputfile=" + outputfile 
#     print(comand)
    os.system(comand)
    
#     os.chdir('cantera')
    return

def printRoP():
    dirOputs = working_dirr +'/'
    species_names =  gas.species_names + surf.species_names
    ## gas 
    omega = np.loadtxt(dirOputs+'omega.dat')
    #surf 
    omegaGasSurf = np.loadtxt(dirOputs+'omegaGasSurf.dat')
    omegaSurf    = np.loadtxt(dirOputs+'omegaSurf.dat')
    omegaS = np.vstack((omegaGasSurf,omegaSurf))
    print('--Surface phase --')
    for i,species in enumerate(species_names):
        print(species,surf.net_production_rates[i],omegaS[i,1])
        
    print('--Surface rate of progress--')
    for i,reac in enumerate(surf.reaction_equations()):
        print(reac,surf.forward_rates_of_progress[i],surf.reverse_rates_of_progress[i])
        
    print('--- reaction constatns ---')
    for i,reac in enumerate(surf.reaction_equations()):
        print(reac,surf.reverse_rate_constants[i],surf.forward_rate_constants[i])
    print('--Gas phase --') 
    for i,species in enumerate(gas.species_names):
        print(species,gas.net_production_rates[i],omega[i,1])
   
    return 


def compareTchemAndCantera(use_yaml=False):
    if use_yaml:
        dirOputs = working_dirr +'/yaml/'
    else:
        dirOputs = working_dirr +'/'
    net_prodGasPhase  = np.loadtxt(dirOputs+'omega.dat') [:,1] 
    
    print('-----------------------------')
    print('Gas phase')
    print('species name cantera  Tchem diff')
    print('-----------------------------')
    for i,species in enumerate(gas.species_names):
        print(species,gas.net_production_rates[i],net_prodGasPhase[i],
              gas.net_production_rates[i]-net_prodGasPhase[i],
             (gas.net_production_rates[i]-net_prodGasPhase[i])/(gas.net_production_rates[i]+1e-23))
    
        if abs((gas.net_production_rates[i]-net_prodGasPhase[i])/gas.net_production_rates[i]) > allow_error:
            print('CHECK this rate')
        print('    ')
        
    return 


 
def getOriginalInput():
    #Reading mass-fraction:
    #======================
    state = np.loadtxt(dirInputs+'inputGasOriginal.dat') # ρ,P,temp, mass fraction
                                  
    Ykk = np.hstack((np.atleast_1d(state[2]),np.atleast_1d(state[1]),state[3:]))
    #Reading site-fraction:
    #======================
    Zkk = np.loadtxt(dirInputs+'inputSurfGasOriginal.dat')
    return Ykk , Zkk

def saveInputsTChem():
    state =  np.hstack((np.atleast_1d(gas.density),np.atleast_1d(Ykk[1]),np.atleast_1d(Ykk[0]),Ykk[2:]))
    # T, P
    np.savetxt(dirInputs + 'inputGas.dat',state)
    
    header_fuel = "T P"
    for sp in gas.species_names:
        header_fuel += " "+sp 
           
    Nvar = 2 + gas.n_species
    sample = np.atleast_2d( np.hstack((np.atleast_1d(Ykk[0]),np.atleast_1d(Ykk[1]),Ykk[2:])) )
#     print(sample)
    np.savetxt(dirInputs +'sample.dat',sample,header=header_fuel,comments='')

    return 

def getRandomSample(Nvars,μ=0,σ=1,sum1=True):
    sample = np.random.rand(Nvars)*σ+μ
    if sum1:
        sample /= np.sum(sample)
    
    return sample

def makeTChemRandomSample():
    oneatm = 1013250.0
    massFraction = getRandomSample(gas.n_species)
    T            = getRandomSample(1,μ=1200,σ=500,sum1=False)
    P            = getRandomSample(1,μ=50*oneatm,σ=2*oneatm,sum1=False) # Pascal
    Ykk = np.hstack((T,P,massFraction))
    return Ykk

### User inputs
* dirInputs : input and/or ouput 
* direx: Where TChem executable (tchem-example-reactionrates-surface.x) is located. source code is at <Tchemrepo>/src/example/TChem_ReactionRatesSurface.cpp
* dirr : run directory     
* allow_error: allow error between cantera and Tchem
* make_random_sample: true or false, if true a random state vector will be used. else a vector will be loaded inputGasOriginal.dat and inputSurfGasOriginal.dat.  

In [3]:
# TchemRepo = '/Users/odiazib/csp_clang_bld_develop/TChemRelease2.1.0'
# TchemRepo = '/Users/odiazib/csp_clang_bld_develop/TChem-Yaml/'
TchemRepo = '/Users/odiazib/csp_clang_bld_develop/TChem++/'
working_dirr      = os.getcwd()#TchemRepo + '/master/src/example/SurfVerification/'

direx     = TchemRepo + '/install/example/'
allow_error        = 1e-3
make_random_sample = True
print('Working directory : ',working_dirr)

Working directory :  /Users/odiazib/data/GasVerification


### With an initial guess

In [4]:
# dirInputs = TchemRepo + '/master/src/example/SurfVerification/data/reaction-rates-surfaces/PT/'
dirInputs = working_dirr + '/inputs/'
gas  = ct.Solution(dirInputs+'ch4NOxRanzi.cti', 'gas')
# surf = ct.Interface(dirInputs+'chem.cti', 'surface1', [gas])

print('Input directory : ', dirInputs)


# dirInputs = TchemRepo + 'build/example/SurfVerification/data/reaction-rates-surfaces/X/'

# dirInputs = TchemRepo + '/master/src/example/SurfVerification/data/reaction-rates-surfaces/X/'
# gas  = ct.Solution(dirInputs+'chem_annotated-gas_surface.cti', 'gas')
# surf = ct.Interface(dirInputs+'chem_annotated-gas_surface.cti', 'surface1', [gas])

if make_random_sample:
    Ykk  = makeTChemRandomSample()
else:    
    Ykk  = getOriginalInput()
#     Ykk , Zkk = getOriginalInput()

gas.Y  = Ykk[2:]
gas.TP = Ykk[0], Ykk[1]

saveInputsTChem()

Input directory :  /Users/odiazib/data/GasVerification/inputs/


In [5]:
runTchem() # run Tchem

In [6]:
print('*-------------*')
print('With an random guess')
compareTchemAndCantera(use_yaml=False)

*-------------*
With an random guess
-----------------------------
Gas phase
species name cantera  Tchem diff
-----------------------------
AR 0.0 0.0 0.0 0.0
    
N2 1429643718.1753345 1429671000.0 -27281.824665546417 -1.908295354899081e-05
    
HE 0.0 0.0 0.0 0.0
    
H2 1093258689.303309 1093249000.0 9689.303308963776 8.862772739669135e-06
    
H -4383986384.388043 -4383810000.0 -176384.38804340363 4.023379011201582e-05
    
O2 384594432.31502455 384609900.0 -15467.68497544527 -4.0218171860521264e-05
    
O -14412378255.952925 -14412460000.0 81744.0470752716 -5.671794454985792e-06
    
H2O 2743268719.583537 2743273000.0 -4280.416462898254 -1.560334367661975e-06
    
OH -917831570.7758124 -917871600.0 40029.22418761253 -4.3612821199620717e-05
    
H2O2 -4262780.381268554 -4262363.0 -417.3812685543671 9.791291861725217e-05
    
HO2 680257349.0118731 680266000.0 -8650.98812687397 -1.2717228471607407e-05
    
CO 3075857793.1477 3075876000.0 -18206.852300167084 -5.9192763529990694e-06
  

/Users/odiazib/opt/miniconda3/envs/cantera3/lib/python3.7/site-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in double_scalars
